In [11]:
# delete English and numbers in the file

import string
import re

fin = open('wiki.zh.txt.jian', 'r')
fout = open('wiki.zh.de_eng.txt', 'w')

for line in fin.readlines():
    fout.write(re.sub('[a-zA-Z0-9]', '', line))
fout.close()

In [ ]:
# divide words using jieba

import jieba

with open('wiki.zh.de_eng.txt', errors='ignore', encoding='utf-8') as fp:
    lines = fp.readlines()
    for line in lines:
        seg_list = jieba.cut(line)
        with open('wiki.zh.divide.txt', 'a', encoding='utf-8') as ff:
            ff.write(' '.join(seg_list))

In [ ]:
# delete multiples spaces

fin = open('wiki.zh.divide.txt', 'r')
fout = open('wiki.zh.final.txt', 'w')

for line in fin.readlines():
    fout.write(' '.join(line.split()))
fout.close()

In [ ]:
# train model and save

from gensim.models import word2vec

sentences = word2vec.Text8Corpus('wiki.zh.final.txt')
model = word2vec.Word2Vec(sentences)
model.save('wiki.zh.model')

In [ ]:
# find the top 15 words that are closest to "他" and "她"

def findStrInFile(str):
    f_pro = open('professions_processed.txt')
    for line in f_pro.readlines():
        l = len(line)
        if str == line[:l - 1]:
            f_pro.close()
            return True
    f_pro.close()
    return False

model = word2vec.Word2Vec.load('wiki.zh.model')
f = open('wiki.zh.he_she.txt', 'w')

f.write("他:\n")
count = 0
for e in model.most_similar(positive=['他'], topn=5000):
    if count < 15 and findStrInFile(e[0]):
        f.write(e[0])
        f.write(" ")
        f.write(str(e[1]))
        f.write("\n")
        count = count + 1
f.write("她:\n")
count = 0
for e in model.most_similar(positive=['她'], topn=5000):
    if count < 15 and findStrInFile(e[0]):
        f.write(e[0])
        f.write(" ")
        f.write(str(e[1]))
        f.write("\n")
        count = count + 1

f.close()